In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
# Load the dataset
df = pd.read_csv('UAV_DATA.csv')  # Replace with your file name

# Ensure timestamps are sorted
df = df.sort_values('timestamp')

# Normalize the features
scaler = MinMaxScaler()
features = df.drop(['label', 'timestamp'], axis=1)
df['label'] = df['label'].map({'benign':0,'jamming':1, 'spoofing':2})  # Adjust columns as needed
labels = df['label']  # Replace with the column indicating labels

features_scaled = scaler.fit_transform(features)

# Create sequences
def create_sequences(data, labels, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(labels[i+seq_length - 1])  # Label for the last element in the sequence
    return np.array(X), np.array(y)

sequence_length = 10
X, y = create_sequences(features_scaled, labels, sequence_length)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Check class distribution
print(df['label'].value_counts())


In [ ]:
import numpy as np
from typing import Callable, Tuple

def GAOA(Materials_no: int, Max_iter: int, fobj: Callable, dim: int, lb: float, ub: float, C3: float, C4: float, max_fes: int) -> Tuple[np.ndarray, float, np.ndarray]:
    # Initialization
    Max_iter = round(max_fes / Materials_no)
    FES = 0
    C1, C2 = 2, 6
    index = 0
    u, l = 0.9, 0.1  # Parameters in Eq. (12)
    
    # Initial positions and other variables
    X = lb + np.random.rand(Materials_no, dim) * (ub - lb)  # Initial positions Eq. (4)
    den = np.random.rand(Materials_no, dim)  # Eq. (5)
    vol = np.random.rand(Materials_no, dim)
    acc = lb + np.random.rand(Materials_no, dim) * (ub - lb)  # Eq. (6)
    
    # Evaluate initial solutions
    Y = np.array([fobj(X[i, :]) for i in range(Materials_no)])
    FES += Materials_no

    # Sort and select the best
    sorted_indices = np.argsort(Y)
    Scorebest = Y[sorted_indices[0]]
    Xbest = X[sorted_indices[0], :]
    den_best = den[sorted_indices[0], :]
    vol_best = vol[sorted_indices[0], :]
    acc_best = acc[sorted_indices[0], :]
    acc_norm = acc.copy()
    Convergence_curve = np.zeros(Max_iter)
    Convergence_curve[index] = Scorebest
    index += 1
    num_agents = int(0.1 * Materials_no)

    # Random walk parameters
    max_step_initial = 0.1 * (ub - lb)
    decay_rate = 2
    neighborhood_size = 5
    t = 1

    while t <= Max_iter and FES < max_fes:
        # Update parameters
        TF = np.exp((t - Max_iter) / Max_iter)  # Eq. (8)
        TF = min(TF, 1)

        max_step = max_step_initial * np.exp(-decay_rate * TF)

        d = np.exp((Max_iter - t) / Max_iter) - (t / Max_iter)  # Eq. (9)
        acc = acc_norm.copy()
        
        # Calculate G, which combines Xbest and the mean of X across all agents
        G = 0.25 * (Xbest + np.tile(np.mean(X[:Materials_no, :], axis=0), (Materials_no, 1)))

        # Neighborhood selection and updates for den, vol, acc
        for i in range(Materials_no):
            neighbors = np.random.choice(np.delete(np.arange(Materials_no), i), neighborhood_size - 1, replace=False)
            neighbor_avg = np.mean(np.vstack([X[neighbors, :], G[i, :]]), axis=0)

            den[i, :] += C1 * np.random.rand() * acc_norm[i, :] * (neighbor_avg - X[i, :]) * d
            vol[i, :] += C1 * np.random.rand() * acc_norm[i, :] * (G[i, :] - X[i, :]) * d

            if TF < 0.5:  # Collision phase
                mr = np.random.randint(Materials_no)
                acc_temp = (den[mr, :] + vol[mr, :] * acc[mr, :]) / (np.random.rand() * den[i, :] * vol[i, :])
            else:  # Non-collision phase
                acc_temp = (den_best + vol_best * acc_best) / (np.random.rand() * den[i, :] * vol[i, :])

            # Normalize acceleration
            acc_norm[i, :] = ((u * (acc_temp - np.min(acc_temp))) / (np.max(acc_temp) - np.min(acc_temp))) + l

        # Random walk step
        if TF > 0.5:
            for i in range(num_agents):
                step_size = max_step * np.random.rand(dim)
                direction = np.random.choice([-1, 1], dim)
                X[i, :] += direction * step_size

        # Boundary check for X
        X = fun_checkpositions(dim, X, Materials_no, lb, ub)

        # Generate new positions Xnew
        Xnew = np.zeros_like(X)
        for i in range(Materials_no):
            if TF < 0.5:
                mrand = np.random.randint(Materials_no)
                Xnew[i, :] = X[i, :] + C1 * np.random.rand() * acc_norm[i, :] * (X[mrand, :] - X[i, :]) * d
            else:
                p = 2 * np.random.rand() - C4
                T = C3 * TF
                T = min(T, 1)
                Xnew[i, :] = Xbest + C2 * np.random.rand() * acc_norm[i, :] * (p * Xbest - X[i, :]) * d

        # Apply boundary check to Xnew
        Xnew = fun_checkpositions(dim, Xnew, Materials_no, lb, ub)

        # Evaluate new solutions and update X, Y
        for i in range(Materials_no):
            v = fobj(Xnew[i, :])
            FES += 1
            if v < Y[i]:
                X[i, :] = Xnew[i, :]
                Y[i] = v

        # Update best score and position if found
        var_Ybest = np.min(Y)
        var_index = np.argmin(Y)
        Convergence_curve[index] = var_Ybest
        index += 1
        if var_Ybest < Scorebest:
            Scorebest = var_Ybest
            Xbest = X[var_index, :]
            den_best = den[var_index, :]
            vol_best = vol[var_index, :]
            acc_best = acc_norm[var_index, :]

        t += 1

    return Xbest, Scorebest, Convergence_curve

def fun_checkpositions(dim: int, vec_pos: np.ndarray, var_no_group: int, lb: float, ub: float) -> np.ndarray:
    """
    Ensure that each position in vec_pos is within the bounds [lb, ub].
    """
    vec_pos = np.clip(vec_pos, lb, ub)
    return vec_pos


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def objective(params):
    """
    Objective function for GAOA. Trains an LSTM model and returns the negative validation accuracy.
    Args:
        params: List of hyperparameters to optimize [num_units, dropout_rate, learning_rate].
    Returns:
        Negative validation accuracy (as GAOA minimizes the objective).
    """
    num_units1 = int(params[0])        # Number of LSTM units
    num_units2 = int(params[1])        # Number of LSTM units
    dropout_rate = np.clip(params[2], 0.0, 1.0)        # Dropout rate
    learning_rate = params[3]        # Learning rate

    # Define LSTM model
    model = Sequential([
        LSTM(num_units1, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(dropout_rate),
        LSTM(num_units2),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')  # Assuming 3 classes
    ])
    
    # Compile model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model (early stopping to prevent overfitting)
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data=(X_val, y_val),callbacks=[early_stopping])
    
    # Evaluate model
    y_pred = model.predict(X_test)
    y_pred_classes = y_pred.argmax(axis=1)
    accuracy = accuracy_score(y_test, y_pred_classes)
    
    return -accuracy 


In [ ]:
dim = 4  # Number of hyperparameters: [num_units1, num_units2, dropout_rate, learning_rate]
lb = np.array([32, 32, 0.1, 1e-4])    # Lower bounds
ub = np.array([64, 64, 0.5, 1e-2])    # Upper bounds

Materials_no = 10  # Number of candidate solutions
Max_iter = 20      # Maximum iterations
C3 = 2             # GAOA-specific parameter
C4 = 0.5           # GAOA-specific parameter
max_fes = 200      # Maximum function evaluations

# Run GAOA
best_params, best_score, convergence_curve = GAOA(
    Materials_no=Materials_no,
    Max_iter=Max_iter,
    fobj=objective,
    dim=dim,
    lb=lb,
    ub=ub,
    C3=C3,
    C4=C4,
    max_fes=max_fes
)

# Output results
print("Best Hyperparameters found by GAOA:", best_params)
print("Best Validation Accuracy (GAOA):", -best_score)


In [ ]:
num_units1 = int(best_params[0])
num_units2 = int(best_params[1])
dropout_rate = best_params[2]
learning_rate = best_params[3]
# Best Hyperparameters found by GAOA: [5.09384143e+01 3.54073999e+01 3.15575453e-01 8.01389167e-03]
# Best Validation Accuracy (GAOA): 0.9983432736911863
# Define the optimized LSTM model

final_model = Sequential([
        LSTM(num_units1, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(dropout_rate),
        LSTM(num_units2),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')  # Assuming 3 classes
    ])

# Compile the optimized model
final_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the optimized model
history = final_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:

y_pred_probs = final_model.predict(X_test)

y_pred = np.argmax(y_pred_probs, axis=1)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Benign", "Jamming", "Spoofing"],
            yticklabels=["Benign", "Jamming", "Spoofing"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Benign", "Jamming", "Spoofing"]))


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

class_names = {0: 'Benign', 1: 'Jamming', 2: 'Spoofing'}

n_classes = len(class_names)
y_test_binarized = label_binarize(y_test, classes=np.arange(n_classes))

fpr = {}
tpr = {}
roc_auc = {}

plt.figure(figsize=(10, 8))

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], label=f"{class_names[i]} (AUC = {roc_auc[i]:.3f})")

fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_pred_probs.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
plt.plot(fpr["micro"], tpr["micro"], linestyle='--', label=f"Micro-average (AUC = {roc_auc['micro']:.3f})")

plt.plot([0, 1], [0, 1], 'k--', label="Chance")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()

conf_matrix = confusion_matrix(y_test, y_pred) 
fpr_class = {}
fnr_class = {}
detection_rate = {}

for i in range(n_classes):
    TP = conf_matrix[i, i]
    FP = conf_matrix[:, i].sum() - TP
    FN = conf_matrix[i, :].sum() - TP
    TN = conf_matrix.sum() - (TP + FP + FN)

    fpr_class[class_names[i]] = FP / (FP + TN) if (FP + TN) > 0 else 0
    fnr_class[class_names[i]] = FN / (FN + TP) if (FN + TP) > 0 else 0
    detection_rate[class_names[i]] = TP / (TP + FN) if (TP + FN) > 0 else 0

for class_name in class_names.values():
    print(f"{class_name}: FPR = {fpr_class[class_name]:.2f}, FNR = {fnr_class[class_name]:.2f}, Detection Rate = {detection_rate[class_name]:.2f}")
